โจทย์นี้จะใกล้ตัวงาน IT Support ที่ต้องดึงข้อมูลมาทำสรุป หรือช่วยดูหลังบ้านให้พ่อค้าแม่ค้าออนไลน์ครับ

สมมติว่าป๋ามีข้อมูล "รายการสั่งซื้อ (Orders)" ที่ต้องจัดการส่งของให้ลูกค้า เราจะใช้ Pandas ช่วยจัดการยังไง มาดูกันครับ

1. สร้างตารางข้อมูลการจัดส่ง (Logistics DataFrame)
สมมติเรามีข้อมูลเลขพัสดุ น้ำหนักของ และสถานะการจัดส่งครับ

In [1]:
import pandas as pd

# จำลองการสั่งซื้อและการขนส่ง
data = {
    'Order_ID': ['ORD-001', 'ORD-002', 'ORD-003', 'ORD-004', 'ORD-005'],
    'Customer': ['ร้านชานม A', 'คุณสมชาย', 'บริษัท B', 'น้องไก่', 'ป้าแมว'],
    'Weight_KG': [1.5, 0.5, 20.0, 2.2, 5.0],      # น้ำหนัก (กก.)
    'Status': ['ส่งแล้ว', 'รอจัดส่ง', 'ส่งแล้ว', 'รอแพ็ค', 'ยกเลิก'],
    'Price': [500, 150, 3000, 750, 1200]           # ราคาสินค้า
}

df = pd.DataFrame(data)
print('--- ตารางรายการสั่งซื้อทั้งหมด ---')
print(df)

--- ตารางรายการสั่งซื้อทั้งหมด ---
  Order_ID    Customer  Weight_KG    Status  Price
0  ORD-001  ร้านชานม A        1.5   ส่งแล้ว    500
1  ORD-002    คุณสมชาย        0.5  รอจัดส่ง    150
2  ORD-003    บริษัท B       20.0   ส่งแล้ว   3000
3  ORD-004     น้องไก่        2.2    รอแพ็ค    750
4  ORD-005      ป้าแมว        5.0    ยกเลิก   1200


2. โจทย์จริง: ตามหาออเดอร์ตกค้าง (Filtering)
สมมติเจ้านายหรือทีมแพ็คของถามป๋าว่า "วันนี้มีออเดอร์ไหนที่ยังไม่ได้ส่งบ้าง?" (คือสถานะไม่ใช่ 'ส่งแล้ว' และไม่ใช่ 'ยกเลิก') หรือเอาแบบง่ายๆ คือหาอันที่ "รอจัดส่ง" หรือ "รอแพ็ค"

ป๋าไม่ต้องมานั่งไล่ดูทีละบรรทัด สั่ง Pandas ทีเดียวจบครับ:

In [2]:
# สร้างเงื่อนไข: เลือกเฉพาะสถานะที่เป็น 'รอจัดส่ง' หรือ 'รอแพ็ค'
# เครื่องหมาย | แปลว่า "หรือ" (OR)
pending_orders = df[(df['Status'] == 'รอจัดส่ง') | (df['Status'] == 'รอแพ็ค')]

print('--- รายการที่ต้องรีบจัดการ ---')
print(pending_orders)

--- รายการที่ต้องรีบจัดการ ---
  Order_ID  Customer  Weight_KG    Status  Price
1  ORD-002  คุณสมชาย        0.5  รอจัดส่ง    150
3  ORD-004   น้องไก่        2.2    รอแพ็ค    750


3. โจทย์จริง: คำนวณค่าส่งตามน้ำหนัก (Calculation)
สมมติบริษัทขนส่งคิดราคา กิโลกรัมละ 20 บาท ป๋าอยากรู้ว่าแต่ละออเดอร์ต้องจ่ายค่าส่งเท่าไหร่ และยอดรวม (ค่าของ + ค่าส่ง) เป็นเงินเท่าไหร่

    ใช้ Pandas คูณเลขทั้งตารางทีเดียวครับ ไม่ต้องกดเครื่องคิดเลข:

In [3]:
# 1. คำนวณค่าส่ง (Shipping_Cost) = น้ำหนัก * 20 บาท
df['Shipping_Cost'] = df['Weight_KG'] * 20

# 2. คำนวณยอดที่ต้องเก็บเงินลูกค้าปลายทาง (Total_Bill) = ราคาสินค้า + ค่าส่ง
df['Total_Bill'] = df['Price'] + df['Shipping_Cost']

print(df[['Order_ID', 'Weight_KG', 'Price', 'Shipping_Cost', 'Total_Bill']])

  Order_ID  Weight_KG  Price  Shipping_Cost  Total_Bill
0  ORD-001        1.5    500           30.0       530.0
1  ORD-002        0.5    150           10.0       160.0
2  ORD-003       20.0   3000          400.0      3400.0
3  ORD-004        2.2    750           44.0       794.0
4  ORD-005        5.0   1200          100.0      1300.0
